<a href="https://colab.research.google.com/github/Gruppe3VDL/Gruppe3VDLws2019/blob/master/exercise2/task2/Task_2_DogBreedsIdentification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

__Complete all sub-tasks marked with ## TO DO! ## and submit the filled notebook on OLAT__ \
__Using a GPU is recommended here__

### Transfer Learning ###
Aim of this notebook is to implement the concept of transfer learning to train a bigger dataset. We try to compete on a well-known competiton on Kaggle known as Dog Breeds Identification. Read more about it here:

https://www.kaggle.com/c/dog-breed-identification/overview



To train a model on the Dog breeds dataset using transfer learning and submit your results to Kaggle.
Note: Below notebook gives some tips to run the code in pytorch. 

In [0]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [78]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [79]:
%cd '/content/drive/My Drive/Task_2/data'

/content/drive/My Drive/Task_2/data


In [4]:
!ls train/

train_images


In [0]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
import shutil

In [0]:
use_cuda = torch.cuda.is_available()

In [0]:
from AlexNet import AlexNet
from train_test import start_train_test

In [0]:
###########################################################################################
## Register on Kaggle With Your repective GroupName  (For example: WS19_VDL_GROUP_01)    ##
##                                                                                       ##
##                                                                                       ##
##                                   ##                                                  ##
##                                  #  #  ###                                            ##
##                                   ##  #                                               ##
##                                 ######                                                ##
##                                #  ##                                                  ##
##                             ###   ##                                                  ##
##                                   ##                                                  ##
##                                ###  ###                                               ##
##                                                                                       ##
##                                                                                       ##
###########################################################################################

In [9]:
###########################################################################################
## Download the Dog-Breeds dataset in folder "data"                                      ##
## from the Kaggle competition link mentioned above                                      ##
##                                                                                       ##
##                                                                                       ##
##                                   ##                                                  ##
##                                  #  #  ###                                            ##
##                                   ##  #                                               ##
##                                 ######                                                ##
##                                #  ##                                                  ##
##                             ###   ##                                                  ##
##                                   ##                                                  ##
##                                ###  ###                                               ##
##                                                                                       ##
##                                                                                       ##
###########################################################################################
# !mkdir /root/.kaggle
# !echo '{"username":"ws19vdlgroup03","key":"820848a6160b4d9eeb38e21cb8ba74c3"}' > /root/.kaggle/kaggle.json
# !pip install kaggle
# !kaggle competitions download -c dog-breed-identification

labels.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test.zip: Skipping, found more recently modified local copy (use --force to force download)
train.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
############################################################################################
## Make your dataset to and dataloaders for the  test data                                ##
############################################################################################
import torchvision

#!unzip 'train.zip'
#!unzip 'test.zip'
#!unzip 'labels.csv.zip'


def load_dataset(src):
    data_path = src
    test_dataset = torchvision.datasets.ImageFolder(
        root=data_path,
        transform=transforms.Compose([
                                      torchvision.transforms.Resize((224, 224)),
                                      torchvision.transforms.ToTensor()])
    )
    test_dataset = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=64,
        num_workers=0,
        shuffle=True
    )
    return test_dataset
   
test_dataset = load_dataset('test/')

In [84]:
####################################################################################################
## TO DO! : Split train data into 20% validation set and make dataloaders for train and val split ##
####################################################################################################
#!mkdir train/train_images
#!mv train/*.jpg train/train_images/ 

#!mkdir test/test_images
#!mv test/*.jpg test/test_images/ 

from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms


data_transforms = transforms.Compose([transforms.ToTensor()])
train_dataset = torchvision.datasets.ImageFolder(root="train/", transform=data_transforms)
train_labels = pd.read_csv('labels.csv')

y_train, y_validation = train_test_split(train_labels, test_size=0.20, random_state=42)

print("Train Set", y_train.shape);
print("Validation Set", y_validation.shape);

Train Set (8177, 2)
Validation Set (2045, 2)


In [0]:
# HINT: 
# One can make their own custom dataset and dataloaders using the CSV file or
# Convert the Dog-breed training dataset into Imagenet Format, where all images of one class are in a
# folder named with class as in the below given format. Standard Pytorch Datasets and Dataloaders can then be applied
# over them
# Root
# |
# |---Class1 ___Img1.png
# |          ___Img2.png
# |
# |---Class2 ___Img3.png
# |          ___Img4.png
# |....
# |....

import os
from PIL import Image
from shutil import copyfile

train_dir = "converted/train/"
valid_dir = "converted/valid/"

ids = train_labels['id'].to_numpy()
breeds = train_labels['breed'].to_numpy()

if not os.path.exists(train_dir):
  os.makedirs(train_dir)

if not os.path.exists(valid_dir):
  os.makedirs(valid_dir)

for i in range(0, len(breeds)):
  if ids[i] in y_train.to_numpy()[:, 0]:
    p = train_dir + breeds[i] + "/"
    if not os.path.exists(p):
      os.makedirs(p)

    # save picture at p
    copyfile("train/train_images/" + ids[i] + ".jpg", p + ids[i] + ".jpg")
  else:
    p = valid_dir + breeds[i] + "/"
    if not os.path.exists(p):
      os.makedirs(p)
    
    # save picture at p
    copyfile("train/train_images/" + ids[i] + ".jpg", p + ids[i] + ".jpg")

__Train famous Alexnet model on Dog breeds dataset. It is not easy to train the alexnet model from 
scratch on the Dog breeds data itself. Curious minds can try for once to train Alexnet from scratch. We adopt Transfer Learning here. We 
obtain a pretrained Alexnet model trained on Imagenet and apply transfer learning to it to get better results.__

## Transfer Learning

In [0]:
#####################################################################################################
## TO DO! :  Freeze the weigths of the pretrained alexnet model and change the last classification ##
## layer from 1000 classes of Imagenet to 120 classes of Dog Breeds, only classification layer     ##
## should be unfreezed and trainable                                                               ##
#####################################################################################################
import torchvision.models as models
import copy

from AlexNet import AlexNet_imagenet

pretrained_alexnet = models.alexnet(pretrained=True)
pretrained_modules = pretrained_alexnet.modules()
wts = []
for i, j in enumerate(pretrained_modules):
  if i in [1, 17, 18, 19, 20, 21, 22]:
    wts += [copy.deepcopy(j.state_dict())]

# best_model_wts = copy.deepcopy(pretrained_alexnet.state_dict())

new_alexnet = AlexNet_imagenet(num_classes=120)
new_modules = new_alexnet.modules()
for i, j in enumerate(new_modules):
  if i is 1:
    j.load_state_dict(wts[0])
  elif i is 16:
    j.load_state_dict(wts[1])
  elif i is 17:
    j.load_state_dict(wts[2])
  elif i is 18:
    j.load_state_dict(wts[3])
  elif i is 19:
    j.load_state_dict(wts[4])
  elif i is 20:
    j.load_state_dict(wts[5])
  elif i is 21:
    j.load_state_dict(wts[6])

# Below function will directly train your network with the given parameters to 5 epochs
# You are also free to use function learned in task 1 to train your model here
criterion = nn.CrossEntropyLoss()

# define train_loader and val_loader of
trainloader = load_dataset('converted/train/')
validloader = load_dataset('converted/valid/')
new_alexnet.cuda()

train_loss, test_loss = start_train_test(new_alexnet, trainloader, validloader, criterion)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:100: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule.See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)



=> Training Epoch #1, LR=0.0100
| Epoch [  1/  5] 		Loss: 4.9038 Acc@1: 0.709%

| Validation Epoch #1			Loss: 4.7907 Acc@1: 0.88%
* Test results : Acc@1 = 0.88%
| Elapsed time : 0:01:04

=> Training Epoch #2, LR=0.0100
| Epoch [  2/  5] 		Loss: 44.7649 Acc@1: 1.015%

| Validation Epoch #2			Loss: 4.7953 Acc@1: 1.03%
* Test results : Acc@1 = 1.03%
| Elapsed time : 0:02:08

=> Training Epoch #3, LR=0.0100
| Epoch [  3/  5] 		Loss: 5.1042 Acc@1: 0.783%

| Validation Epoch #3			Loss: 4.7692 Acc@1: 1.03%
* Test results : Acc@1 = 1.03%
| Elapsed time : 0:03:12

=> Training Epoch #4, LR=0.0100


## Making Kaggle Submission

In [0]:
##############################################################################################################
## Not So optimal Code: This can take upto 2 minutes to run: You are free to make an optimal version :)     ##
## It iterates over all test images to compute the softmax probablities from the last layer of the network  ##
##############################################################################################################
from transform import transform_testing
import PIL.Image
import torch.nn.functional as F
import numpy as np


augment_image = transform_testing()
test_data_root = 'data/dog_breeds/test/' 
test_image_list = os.listdir(test_data_root) # list of test files 
result = []
for img_name in test_image_list:
    img = PIL.Image.open(test_data_root + img_name)
    img_tensor = augment_image(img)
    with torch.no_grad():
        output = pretrained_resnet(img_tensor.unsqueeze_(0).cuda())
        probs = F.softmax(output, dim=1)
    result.append(probs.cpu().numpy())

all_predictions = np.concatenate(result)
print(all_predictions.shape)

In [0]:
df = pd.DataFrame(all_predictions)
file_list = os.listdir('data/dog_data_imagenet/train') # list of classes to be provided here
df.columns = sorted(file_list)

# insert clean ids - without folder prefix and .jpg suffix - of images as first column
test_data_root = 'data/dog_breeds/test/' # list of all test files here
test_image_list = os.listdir(test_data_root)
df.insert(0, "id", [e[:-4] for e in test_image_list])
df.to_csv(f"sub_1_alexnet.csv", index=False)

### TO DO!: ###
Submit the created CSV file to Kaggle, with a score(cross entropy loss) not more than __2.0__\
Take a snapshot of your rank on Kaggle Public Leaderboard and include the image here ...
For example :
![title](https://github.com/Gruppe3VDL/Gruppe3VDLws2019/blob/master/snp2.png?raw=1)

## CHALLENGE  (optional)
Compete against each other, Come up with creative ideas. Try beating the score of __0.3__. The group with minimum score gets a small prize at the time when the solutions are discussed. 


__Hints:__

1. Instead of Alexnet use pretrained resnet 18 model for better accuracy
2. Instead of a just adding the last classification layer, try adding two layers to get a better loss
3. Train some more layers at the end of the network with a very very small learning rate
4. Add Batch Normalizations or Dropout to the layers you have added, (If not present)
5. Add more augmentation to your dataset, see tranform.py file and use auto autoaugment to apply more rigorous data augmentation techniques